In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u

%matplotlib inline

This notebook describes how to set up a grain size distribution: `newdust.graindist.GrainDist`. Grain size distributions combine the size distribution (number density distribution of dust grains with different radii, $a$) with the material properties of the grain (such as `rho`, provided by `newdust.graindist.composition`. It does not matter what the composition of the dust grains are, except for the material density of those grains.

In [ ]:
from newdust import graindist

In [ ]:
# Some generic parameters
AMAX = 0.3 # maximum grain size, in um
RHO  = 3.0 # grain material density, in g cm^-3
MD   = 1.e-5 # dust mass column, g cm^-2

## Making a GrainDist object from scratch

There are a few shortcuts for setting up grain distributions (which connect a size distribution function in the `newdust.graindist.sizedist` module with the material properties of the grain, such as density, in th `newdust.graindist.composition` module). Here, we will show how to set them up together by hand.

First, one must choose a function for the grain size distribution. There are three built-in `newdust.sizedist` classes:

* **Grain** (`newdust.graindist.sizedist.Grain`): A single grain size distribution consisting solely of grains with radius `a`
* **Powerlaw** (`newdust.graindist.sizedist.Powerlaw`): A power law distribution of grains with radii between `amin` and `amax` and a power law slope of negative `p`. Use the keyword `na` to set the number of grid points to use for the grain radii, `a`.
* **ExpCutoff** (`newdust.graindist.sizedist.ExpCutoff`): A power law with an exponential cutoff, which describes a continuous distribution of grains with radii starting with `amin` and following a power law slope of negative `p`. The radius `acut` is used to taper the grain size distribution on the large end of the distribution. This grain size distibution follows the formula:
$$ \frac{dn}{da} \propto a^{-p} \exp(-a/a_{\rm cut})$$
The maximum grain size evaluated for the distribution is `nfold` $\times$ `acut`

Continuous distributions (`Powerlaw` and `ExpCutoff`) have units of `mdens` in g cm$^{-2}$ $\mu$m$^{-1}$ and `ndens` in cm$^{-2}$ $\mu$m$^{-1}$. The single grain size distribution, `Grain` has units of `mdens` in g cm$^{-2}$ and `ndens` in cm$^{-2}$.

The input parameter `md` provides the total integrated dust mass column in g cm$^{-2}$. The grain size distributions are automatically normalized so that `mdens` will integrate over the grain radius values to return `md`.

In [ ]:
# To completely customize your grain size distribution, 
# you must first set up a newdust.sizedist object

# This only provides the functional form of the grain size distribution
# It does not connect to the composition or density of the dust grains
MRN  = graindist.sizedist.Powerlaw(amax=AMAX)
ECUT = graindist.sizedist.ExpCutoff(acut=AMAX)

# Next, you much specify a composition for the grains
# Composition provides the material density, `rho`, which is needed to translate
# dust mass column into number density of dust grains
SIL  = graindist.composition.CmSilicate(rho=RHO)

In [ ]:
# Here we combine the sizedist and composition objects to define a GrainDist object
gd_mrn  = graindist.GrainDist(MRN, SIL, md=MD, custom=True) # typical powerlaw (MRN)
gd_ecut = graindist.GrainDist(ECUT, SIL, md=MD, custom=True) # MRN with an exponential cut-off

In [ ]:
# Use the built-in plotting function to visualize the results
# It is conventional to plot the grain size distribution as (dn/da) a^4
# Particularly useful for X-ray studies, because the scattering cross-section is propto a^4
ax = plt.subplot(111)
gd_mrn.plot(ax, color='k', lw=2, label='Powerlaw')
gd_ecut.plot(ax, color='b', lw=2, alpha=0.8, label='Exponential cut-off')
plt.legend(loc='upper left', frameon=False)

In [ ]:
# Or, you could plot the properties by hand to get rid of the a^4 component
plt.plot(gd_mrn.a, gd_mrn.ndens, 'k-', lw=2, label='Powerlaw')
plt.plot(gd_ecut.a, gd_ecut.ndens, 'b-', lw=2, alpha=0.8, label='Exponential cut-off')
plt.xlabel(gd_mrn.a.unit)
plt.ylabel(r'$dn/da$ (cm$^{-2}$ um$^{-1}$)')
plt.loglog()
plt.legend()

In [ ]:
## Note that the grain radii have units on them... 
print(gd_mrn.a)

In [ ]:
# ... but the number densities do not
print(gd_mrn.ndens)

## Making a GrainDist object with helper function

Instead of supplying the `newdust.graindist.sizedist` and `newdust.graindist.composition` objects directly, you can provide strings. 

Options for the size distribution function:
+ "Grain"
+ "Powerlaw"
+ "ExpCutoff"

Options for the composition:
+ "Silicate" - uses Draine (2003) silicate properties
+ "Graphite" - uses Draine (2003) graphite properties, perpendicular orientation
+ "Drude" - uses the Drude approximation

You can also use the keywords `amax` and `rho` to customize. The `rho` keyword value is used to initiate the composition objct. The `amax` keyword value is used to initiate the size distribution function, and acts differently depending on the size distribution you picked:
+ Grain: amax sets the singular grain size
+ Powerlaw: amax sets the maximum grain size in the distribution
+ ExpCutoff: amax sets the `acut` value

Any other keywords provided to the `newdust.graindist.GrainDist` are used to initiate the size distribution.

In [ ]:
# To use the shortcuts, provide the name of the sizedist class you want to use, as a string
# You can also provide the grain composition as a string: 'Silicate', 'Graphite', or 'Drude'
gd_mrn2 = graindist.GrainDist('Powerlaw', 'Silicate', amax=AMAX, rho=RHO, md=MD)
gd_ecut2 = graindist.GrainDist('ExpCutoff', 'Silicate', amax=AMAX, rho=RHO, md=MD)

In [ ]:
ax = plt.subplot(111)

# We still get the same power law as before
gd_mrn.plot(ax, color='k', lw=2, label='')
gd_mrn2.plot(ax, color='r', lw=2, ls='--', label='Powerlaw')

# And we get the same ExpCutoff distribution as before
gd_ecut.plot(ax, color='k', alpha=0.5, lw=2, label='')
gd_ecut2.plot(ax, color='b', lw=2, ls='--', label='Exponential Cutoff')

plt.legend(loc='upper left', frameon=False)

### Note that the values used in this example are different from the defaults

Silicate has a default grain material density of 3.8 g cm$^{-3}$. If you don't define `rho`, there is a default value for each.

In [ ]:
# Set up the same size distributions, but fall back on the default silicate density
gd_mrn3 = graindist.GrainDist('Powerlaw', 'Silicate', md=MD)
gd_ecut3 = graindist.GrainDist('ExpCutoff', 'Silicate', md=MD)

In [ ]:
# Plot to compare the results of using a slightly larger `rho` (the default)
ax = plt.subplot(111)
gd_mrn.plot(ax, color='k', lw=2, label='')
gd_mrn3.plot(ax, color='r', lw=2, ls='--', label='Powerlaw')

gd_ecut.plot(ax, color='k', lw=2, label='')
gd_ecut3.plot(ax, color='b', lw=2, ls='--', label='Exponential Cutoff')
plt.legend(loc='upper left', frameon=False)

# The default grain density is larger than 3.0 g cm^-3, 
# so the number density with the default `rho` is smaller
# (fewer grains needed to obtain the same dust mass)

### Hybride method for initiating a Grain Distribution

As described above, most of the keyword arguments provided to `newdust.graindist.GrainDist` are passed to the size distribution function. In the example below, I set up a power law distribution that extends to larger dust grain sizes than the default MRN function. I set the amax value to a number that is two times larger than the maximum value in `gd_mrn`.

I also change the number of grain radii used to sample the distribution using the keyword `na` (i.e., the number of grain radius $a$ values used to evaluate the size distribution function). I also make it so that the size distribution is uniformly sampled over log-space (instead of linear) by setting the keyword `log=True`.

The plot below shows how it compares to the original `gd_mrn` grain distribution. I plot each sample point in the size distribution with an 'o', to demonstrate the difference in sampling method.

In [ ]:
gd_powerlaw2 = graindist.GrainDist('Powerlaw', 'Silicate', 
                                   md=MD, rho=RHO, amax=2.0*AMAX, na=10, log=True)

ax = plt.subplot(111)
gd_mrn.plot(ax, marker='o', color='k', label='MRN')
gd_powerlaw2.plot(ax, marker='o', color='b', label='Powerlaw2')
plt.loglog()
plt.legend()

Note that the "Powerlaw2" distribution follows the same slope, but extends to largr radii, and overall there are fewer dust grains of each radius. That's because both distributions  integrate to the same total mass.

## Look at the mass density distribution of dust grains

The `md` attribute contains the total dust mass, but the `mdens` attribute (for continuous distributions like `Powerlaw` and `ExpCutoff`) has units of g cm$^{-2}$ um$^{-1}$ and will integrate to the total mass column.

In [ ]:
# Plot mdens to show that it is a continuous distribution
plt.plot(gd_mrn.a, gd_mrn.mdens, 'k-', label='Powerlaw')
plt.plot(gd_ecut.a, gd_ecut.mdens, 'b-', label='ExpCutoff')
plt.loglog()
plt.xlabel('Radius (um)')
plt.ylabel('Mass density (g cm$^{-2}$ um$^{-1}$)')
plt.legend(loc='upper right', frameon=False)

### Single grain size plots are un-informative

You can plot them, but it's just a single point! Also nthe units displayed in the plot are technically wrong. The `ndens` units for a `newdust.graindist.sizedist.Grain` object is number density ($cm^{-2}$) while continuous size distributions have units of $cm^{-2}$ $\mu$m$^{-1}$, because they are meant to be integrated.

In [ ]:
gd_single = graindist.GrainDist('Grain', 'Silicate', amax=AMAX)

ax = plt.subplot(111)
gd_single.plot(ax, marker='o')